# **Assignment 2 - Kaggle**


# **Abstract** : The goal of this notebook is to understand the dataset for the kaggel competition - 'Tabular Playground Series - May 2021' and predict using linear regression, random forest and SHAP values. The original dataset deals with predicting the category on an eCommerce product given various attributes about the listing. But in this dataset all the features are given without names. 
# Initially I performed data analysis to identify and visulise the factors that contribute to target varible. Then I used linear regression and random forest to predict the target value. Accuracy of the model using random forest is 56.9% 

**1**. **Importing** **dataset** **and required libraries**

In [ ]:
import pandas as pd
import numpy as np

train_url = 'https://raw.githubusercontent.com/Mayuri2711/DSME_DataFiles/main/Assignment_2/train.csv'
train_df = pd.read_csv(train_url)


test_url = 'https://raw.githubusercontent.com/Mayuri2711/DSME_DataFiles/main/Assignment_2/test.csv'

test_df = pd.read_csv(test_url)
train_df



,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0,Class_2
1,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
2,2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,1,0,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0,Class_1
3,3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,Class_4
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,Class_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,5,0,0,0,0,0,0,0,0,1,0,0,6,0,0,3,0,0,0,Class_1
99996,99996,0,0,0,0,0,0,0,1,2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,3,0,1,0,0,0,7,0,0,0,2,0,0,0,10,0,0,0,Class_2
99997,99997,0,0,0,0,0,1,0,0,0,4,0,0,3,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,4,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,2,0,Class_3
99998,99998,0,0,0,0,1,0,0,0,0,5,0,0,0,0,2,0,0,0,3,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,4,0,Class_2


In [ ]:
test_df

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,100000,0,0,0,0,0,0,4,4,0,0,5,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0
1,100001,0,0,1,0,0,0,2,0,7,7,0,0,0,0,0,3,0,0,2,0,0,0,0,0,0,0,4,0,2,0,0,2,0,3,6,3,0,0,6,0,3,0,1,0,0,0,1,0,2,1
2,100002,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,0,0,0,0,4,4,0,0,0,0,0,3,0,0,0,0,0,2,11,0,0,0,0,1,0,0,0,0,6,0
3,100003,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,4,1,0,0,1,0,0,1,1,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,6,9,14,3
4,100004,0,0,0,0,0,0,1,0,4,0,0,1,0,0,9,0,0,0,0,0,0,3,0,0,0,0,0,0,2,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,149995,0,0,0,0,0,0,0,1,5,0,2,0,0,0,7,5,0,0,0,0,6,1,0,0,7,2,0,0,2,0,0,0,3,0,0,0,0,1,0,1,0,1,0,0,0,3,0,0,3,0
49996,149996,0,0,0,0,0,0,10,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,11,0,0,0,0,1,0,0,0,0,0,3
49997,149997,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,18,0,0,0,0,0,2,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49998,149998,0,0,0,0,1,0,0,0,0,4,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,6,0,0,1,0,0,0,0,0,0,7,0,1,0,0,0,0,0,0,0,0,1


**2. Understanding the data**

In [ ]:
# Understanding the datatypes for data cleaning.
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 52 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          100000 non-null  int64 
 1   feature_0   100000 non-null  int64 
 2   feature_1   100000 non-null  int64 
 3   feature_2   100000 non-null  int64 
 4   feature_3   100000 non-null  int64 
 5   feature_4   100000 non-null  int64 
 6   feature_5   100000 non-null  int64 
 7   feature_6   100000 non-null  int64 
 8   feature_7   100000 non-null  int64 
 9   feature_8   100000 non-null  int64 
 10  feature_9   100000 non-null  int64 
 11  feature_10  100000 non-null  int64 
 12  feature_11  100000 non-null  int64 
 13  feature_12  100000 non-null  int64 
 14  feature_13  100000 non-null  int64 
 15  feature_14  100000 non-null  int64 
 16  feature_15  100000 non-null  int64 
 17  feature_16  100000 non-null  int64 
 18  feature_17  100000 non-null  int64 
 19  feature_18  100000 non-n

**3. Finding Null Values for Data Cleaning**

In [ ]:
# Finding out null values column in the overall data
nanValues = train_df.isna()
nanColumns = nanValues.any()

columnsWithNan = train_df.columns[nanColumns].tolist()
print(columnsWithNan)

[]


**Decision for Null values -  As no null or missing values are present in the data data is cleaned.**

**4a. Finding unique values of categorical target variable.**

In [ ]:
# finidng out unique values of target column beacause its dtype is object.
train_df.target.unique()

array(['Class_2', 'Class_1', 'Class_4', 'Class_3'], dtype=object)

**4b. As target is categorical variable using label encoding to convert it to machine readble form. Craeting new varible "target_cat" with label encoding and dropping "target" variable.**

In [ ]:
train_df1= train_df.copy()
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train_df1['target_cat'] = labelencoder.fit_transform(train_df['target'])

train_df1 = train_df1.drop('target', 1)
train_df1

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target_cat
0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0,1
1,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,1,0,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0,0
3,3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,3
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,5,0,0,0,0,0,0,0,0,1,0,0,6,0,0,3,0,0,0,0
99996,99996,0,0,0,0,0,0,0,1,2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,3,0,1,0,0,0,7,0,0,0,2,0,0,0,10,0,0,0,1
99997,99997,0,0,0,0,0,1,0,0,0,4,0,0,3,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,4,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,2,0,2
99998,99998,0,0,0,0,1,0,0,0,0,5,0,0,0,0,2,0,0,0,3,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,4,0,1


In [ ]:
train_df1.describe()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target_cat
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.00000,100000.000000
mean,49999.500000,0.257830,0.431720,0.114130,0.590550,0.599770,0.160820,0.731490,1.228920,0.903350,0.940120,0.313680,0.403750,0.461470,0.07565,2.597320,1.242030,0.364830,0.740000,0.471220,1.564780,0.242890,0.661970,0.184740,0.388040,1.04013,0.408630,0.353550,0.300680,1.106060,0.169600,0.294500,1.372040,0.277510,0.612460,1.122150,0.908980,0.116530,0.398310,2.997170,0.674640,0.712270,0.582070,0.529230,0.616310,0.135100,0.358660,0.516810,0.39004,0.970850,0.55712,1.381160
std,28867.657797,0.929033,1.977862,0.519584,1.844558,2.785531,0.601149,2.343465,2.692732,3.415258,1.904172,1.121168,1.011418,1.128729,0.35901,4.655092,2.160111,1.284152,1.819388,1.096423,4.749449,0.927461,2.182703,0.553854,1.215796,2.53991,1.321669,1.491937,1.592092,1.879629,0.739582,1.352963,3.319325,1.460705,2.283595,2.493045,2.952305,0.411014,1.095268,5.386551,3.732428,1.721863,2.003114,2.300826,2.360955,0.627592,1.464187,2.171415,1.48735,2.576615,1.68093,0.810891
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-2.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,-8.000000,-5.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
25%,24999.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,1.000000
50%,49999.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,1.000000
75%,74999.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,3.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.

In [ ]:
train_df1 = train_df1.drop(columns= "id", axis=1)
train_df1

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target_cat
0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0,1
1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,1,0,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0,0
3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,3
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,5,0,0,0,0,0,0,0,0,1,0,0,6,0,0,3,0,0,0,0
99996,0,0,0,0,0,0,0,1,2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,3,0,1,0,0,0,7,0,0,0,2,0,0,0,10,0,0,0,1
99997,0,0,0,0,0,1,0,0,0,4,0,0,3,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,4,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,2,0,2
99998,0,0,0,0,1,0,0,0,0,5,0,0,0,0,2,0,0,0,3,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,4,0,1


**5a. Feature selection : using Correlation matrix**

In [ ]:
import matplotlib.pyplot as plt
train_df2= train_df1.copy()

corr = train_df2.corr()
correlation = corr.drop('target_cat', axis=1).corrwith(corr.target_cat).sort_values(axis=0, ascending=True)
correlation

feature_37   -0.071893
feature_6    -0.070731
feature_0    -0.062530
feature_29   -0.061734
feature_14   -0.059683
feature_15   -0.057791
feature_25   -0.054968
feature_28   -0.053208
feature_16   -0.053013
feature_20   -0.052900
feature_31   -0.052112
feature_9    -0.051045
feature_10   -0.049315
feature_24   -0.048124
feature_19   -0.047571
feature_18   -0.046740
feature_45   -0.045727
feature_46   -0.045595
feature_35   -0.044611
feature_32   -0.040861
feature_7    -0.039761
feature_23   -0.039284
feature_41   -0.038844
feature_17   -0.038726
feature_40   -0.035713
feature_21   -0.029420
feature_5    -0.029226
feature_22   -0.027954
feature_49   -0.027795
feature_11   -0.026837
feature_3    -0.026728
feature_12   -0.026533
feature_48   -0.025539
feature_4    -0.025510
feature_42   -0.025246
feature_38   -0.024853
feature_1    -0.024662
feature_33   -0.022188
feature_8    -0.020446
feature_30   -0.017860
feature_27   -0.017399
feature_47   -0.012527
feature_26   -0.008309
feature_39 

In [ ]:
import matplotlib.pyplot as plt
train_df2= train_df1.copy()

corr = train_df2.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=0, vmax=0.5).highlight_null(null_color='#f1f1f1').set_precision(2))

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target_cat
feature_0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_1,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_2,-0.01,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_3,0.01,0.00,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_4,-0.00,0.01,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_5,-0.00,0.00,0.01,-0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_6,0.00,0.01,0.00,0.01,0.00,-0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_7,0.01,0.01,0.01,0.01,0.01,0.01,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_8,-0.00,0.01,0.00,0.01,0.01,-0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature_9,0.01,-0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


**Observation** :- Highly correlated features are : 
feature_26   -0.007869 

---


feature_39   -0.006600

---


feature_44   -0.004499

---





feature_43   -0.001195

---


feature_13    0.002566

---


feature_2     0.003193

---


feature_36    0.008416

---


feature_34    0.009036

**5b. Feature selection : - using Random Forest**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
train_df3 = train_df2.copy()

In [ ]:
train_df4 = train_df3.copy()
Y_train = train_df4[['target_cat']]


X_train = train_df4.drop(columns= ['target_cat'])
X_train.head()
Y_train.head()

,target_cat
0,1
1,0
2,0
3,3
4,1


In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_from_model.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


SelectFromModel(estimator=RandomForestClassifier())

In [ ]:
sel.get_support()


array([False, False, False,  True, False, False, False,  True,  True,
        True, False, False,  True, False,  True,  True, False,  True,
        True,  True, False,  True, False, False,  True, False, False,
       False,  True, False, False,  True, False, False,  True,  True,
       False, False,  True, False,  True,  True, False, False, False,
       False, False, False,  True,  True])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
selected_feat = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(selected_feat)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = selected_feat, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)
print(selected_feat)

Index(['id', 'feature_3', 'feature_7', 'feature_8', 'feature_9', 'feature_12',
       'feature_14', 'feature_15', 'feature_17', 'feature_18', 'feature_19',
       'feature_21', 'feature_24', 'feature_28', 'feature_31', 'feature_34',
       'feature_35', 'feature_38', 'feature_40', 'feature_41', 'feature_48',
       'feature_49'],
      dtype='object')


**6.Distrubtions of the independent variables(Highly correlated)**

In [ ]:
## Plot distplot for continuous data columns 
import matplotlib.pyplot as plt
import seaborn as sns
train_df6 = train_df5.copy()

sns.distplot(df4.feature_34, kde='true')
sns.displot(df4.feature_36, kde='true')
sns.displot(df4.feature_13, kde='true')
sns.displot(df4.feature_2, kde='true')
sns.displot(df4.feature_43, kde='true')
sns.displot(df4.feature_44, kde='true')
sns.displot(df4.feature_39, kde='true')

**7. Finding out outliers**

In [ ]:
train_df7 = train_df6.copy()
# Box Plot
import seaborn as sns
sns.boxplot(train_df7['feature_36'])

In [ ]:

import seaborn as sns
sns.boxplot(train_df7['feature_34'])





In [ ]:
sns.boxplot(train_df7['feature_13'])


In [ ]:
sns.boxplot(train_df7['feature_2'])


In [ ]:
sns.boxplot(train_df7['feature_43'])


In [ ]:
sns.boxplot(train_df7['feature_44'])


In [ ]:
sns.boxplot(train_df7['feature_39'])

**contributions statement :- The data for most of the highly correlated features is not normally ditributed. The frequency of the is most at '0' value. As Q1 and Q3 are  0 here IQR will also be 0 for most of the features. hence no use of using IQR or Z score for removing outliers.**

**7c. Instead normalising the data using Power Transformer.**

In [ ]:
train_df8 = train_df7.copy()
test_df2 = test_df1.copy()



selected_Columns = train_df8[['feature_36', 'feature_34', 'feature_13', 'feature_2', 'feature_43',
       'feature_44', 'feature_39']]
selected_Columns_RF = train_df8[['feature_3', 'feature_7', 'feature_8', 'feature_9', 'feature_12',
       'feature_14', 'feature_15', 'feature_17', 'feature_18', 'feature_19',
       'feature_21', 'feature_24', 'feature_28', 'feature_31', 'feature_34',
       'feature_35', 'feature_38', 'feature_40', 'feature_41', 'feature_48',
       'feature_49']]
      

X=selected_Columns.copy()

X_RF = selected_Columns_RF


y = train_df8[['target_cat']]



X.hist()


In [ ]:
y.hist()

In [ ]:
X_test.hist()

In [ ]:
y_test.hist()

**After analyzing above graphs it is evident that Test and Train data are same**

**Citations : - reference from public kaggle kernal about power transformer.**

In [ ]:
train_df8 = train_df7.copy()

from sklearn.preprocessing import PowerTransformer
X[X.columns] = PowerTransformer().fit_transform(X)
X.head()

**8a. Linear Regression Model using feature selection using correlation**




In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train,y_train)


lin_reg.score(X,y) 


**Linear Model accuracy is 0.0011 which is very low. Hence using another feature selecion method - random forest**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_RF, y, test_size=0.2, random_state=7)

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train,y_train)


lin_reg.score(X_RF,y) 


**Slightly incresed accuracy with this feature selection but still not significant hence using another algorithm for better prediction**

**9a. Random Forest model with feature selection using correlation**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

lin_reg = RandomForestClassifier(random_state=42)
lin_reg.fit(X_train,y_train)


lin_reg.score(X,y) 


**Accuracy with Random Forest is 0.58959 which is good.**

9b. Random Forest model with feature selection using random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_RF, y, test_size=0.2, random_state=7)

lin_reg = RandomForestClassifier(random_state=42)
lin_reg.fit(X_train,y_train)


lin_reg.score(X_RF,y) 


**Accuracy is 0.9 which is overfitting the model.** 

**Conclusion :- After analyzing above data , it is best to use random forest algorithm with highly correlated feature selection.**

**10. Applying model on test data**

In [ ]:
test_df2 = test_df.copy()
selected_Columns = test_df2[['feature_36', 'feature_34', 'feature_13', 'feature_2', 'feature_43',
       'feature_44', 'feature_39']]
test_df3 =selected_Columns.copy()
from sklearn.preprocessing import PowerTransformer

powerTransfomerFit = PowerTransformer().fit(X)
X[X.columns] = powerTransfomerFit.transform(X)
test_df3 = powerTransfomerFit.transform(test_df3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
from sklearn.ensemble import RandomForestClassifier

randomForest = RandomForestClassifier(random_state=42)
randomForest.fit(X_train, y_train)
randomForest.score(X_test, y_test)

In [ ]:
randomForest.score(X_test, y_test)

**10. Prediction**

In [ ]:
row_to_show = 5
data_for_prediction = X_test.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)


randomForest.predict_proba(data_for_prediction_array)

**11. SHAP values**

In [ ]:
!pip install shap
import shap # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(randomForest)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)


In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

In [ ]:
from google.colab import files
submission = pd.read_csv("https://raw.githubusercontent.com/Mayuri2711/DSME_DataFiles/main/Assignment_2/submission.csv")
submission['Class_1']=predicts[:,0]
submission['Class_2']=predicts[:,1]
submission['Class_3']=predicts[:,2]
submission['Class_4']=predicts[:,3]

submission.to_csv('submission-tabular-playground-series-may-random-forest-30-05.csv', index = False)
files.download('submission-tabular-playground-series-may-random-forest-30-05.csv')

submission.head()




**Conclusion :-**

---
1. Models used : - Linear regression and random forest.
2. Effective Model :- random forest
3. Feature selction methods used :- Random Forest and Correlation
4. Effective Feature selection method :- Correlation when used with random forest algorithm.
5. Accuracy - 56.9%
6. **What is different than public kernals**:- 1)used two algorithms for prediction and 2 feature selection methods. Out of them used best fitted methods and algorithms for presiction. 
2) Interpreted using SHAP values.

**Problem with the data :-**

---

1. Not normally distributted.
2. As the names of the features were not given it was difficult to understand the meaning of frequently occuring '0' value.



**The MIT License**

Copyright

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

